<a href="https://colab.research.google.com/github/vatsaaa/mtech/blob/main/semester_1/03_assignments/mfml/Assignment_01_Q02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Consider a symmetric and positive definite matrix A of size n `×` n. Without using external libraries like numpy, pandas, sympy etc., write python code to construct an elementary matrix for every elementary row operation that is performed on A so as to get to inverse of matrix A.

In [ ]:
import sys

EPSILON = 100000 * sys.float_info.epsilon

A = [[4, 1, 2],
     [1, 3, 2],
     [2, 2, 5]]

def print_matrix(matrix):
    print("[", end="")
    for i, row in enumerate(matrix):
        updated_row = [0 if abs(element) < EPSILON else element for element in row]
        print(updated_row, end="")
        if i != len(matrix) - 1:
            print(",\n", end="")
    print("]")


In [ ]:
def find_inverse(matrix):
    """Finds the inverse of a square matrix using Gaussian elimination."""

    n = len(matrix)

    # Check if the matrix is square
    if n != len(matrix[0]):
        raise ValueError("Matrix must be square to have an inverse")

    # Create augmented matrix with identity matrix on the right
    augmented_matrix = [row + [0.0] * n for row in matrix]
    for i in range(n):
        augmented_matrix[i][n + i] = 1.0

    # Perform Gaussian elimination
    for j in range(n):
        # Find the pivot row
        pivot_row = j
        for i in range(j + 1, n):
            if abs(augmented_matrix[i][j]) > abs(augmented_matrix[pivot_row][j]):
                pivot_row = i

        # Swap rows if necessary
        if pivot_row != j:
            augmented_matrix[j], augmented_matrix[pivot_row] = augmented_matrix[pivot_row], augmented_matrix[j]

        # Perform row operations to eliminate elements below the pivot
        pivot = augmented_matrix[j][j]
        for i in range(j + 1, n):
            factor = augmented_matrix[i][j] / pivot
            for k in range(j, n * 2):
                augmented_matrix[i][k] -= factor * augmented_matrix[j][k]

    # Check for singular matrix
    for j in range(n):
        if abs(augmented_matrix[j][j]) < 1e-10:
            raise ValueError("Matrix is singular and does not have an inverse")

    # Perform back-substitution to make the diagonal elements equal to 1
    for j in range(n - 1, -1, -1):
        pivot = augmented_matrix[j][j]
        for k in range(n * 2):
            augmented_matrix[j][k] /= pivot
        for i in range(j - 1, -1, -1):
            factor = augmented_matrix[i][j]
            for k in range(n * 2):
                augmented_matrix[i][k] -= factor * augmented_matrix[j][k]

    # Extract the inverse matrix from the right side of the augmented matrix
    inverse = [[augmented_matrix[i][n + j] for j in range(n)] for i in range(n)]

    return inverse


In [ ]:
import numpy as np

print_matrix(np.linalg.inv(A))

[[0.3142857142857143, -0.02857142857142858, -0.11428571428571428],
[-0.02857142857142857, 0.45714285714285713, -0.17142857142857143],
[-0.11428571428571428, -0.1714285714285714, 0.3142857142857143]]


In [ ]:
print_matrix(find_inverse(A))

[[0.3142857142857143, -0.028571428571428567, -0.11428571428571428],
[-0.02857142857142857, 0.45714285714285713, -0.17142857142857143],
[-0.1142857142857143, -0.17142857142857143, 0.3142857142857143]]


In [ ]:
result = np.matmul(A, find_inverse(A))

In [ ]:
print_matrix(result)

[[1.0, 0, 0],
[0, 0.9999999999999999, 0],
[0, 0, 1.0]]


In [ ]:
def find_inverse_with_elementary_matrices(matrix):
    """
    Finds the inverse of a symmetric and positive definite matrix using elementary matrices.

    Args:
        matrix: A symmetric and positive definite matrix of size n x n.

    Returns:
        A dictionary containing:
            - "input_matrix": The original input matrix (unmodified).
            - "inverse_matrix": The inverse of the input matrix.
            - "elementary_matrices": The list of elementary matrices used in the process.

    Raises:
        ValueError: If the input matrix is not square, symmetric, or positive definite.
    """

    n = len(matrix)

    # Check if the matrix is square
    if n != len(matrix[0]):
        raise ValueError("Input matrix must be square")

    # Check if the matrix is symmetric
    for i in range(n):
        for j in range(i + 1, n):
            if abs(matrix[i][j] - matrix[j][i]) > EPSILON:
                raise ValueError("Input matrix must be symmetric")

    # Check if the matrix is positive definite
    for i in range(n):
        if matrix[i][i] <= 0:
            raise ValueError("Input matrix must be positive definite")

    # Initialize the inverse matrix as the identity matrix
    inverse = [[1 if i == j else 0 for j in range(n)] for i in range(n)]

    # Initialize the list of elementary matrices
    elementary_matrices = []

    # Create a copy of the matrix to avoid modifying the original input
    matrix_copy = [row.copy() for row in matrix]

    # Perform Gaussian elimination with row operations on the copy of the matrix
    for i in range(n):
        # Find the pivot element
        pivot = matrix_copy[i][i]

        # Check if the pivot is close to zero
        if abs(pivot) < EPSILON:
            raise ValueError("Input matrix is not positive definite")

        # Update the inverse matrix
        for j in range(n):
            inverse[i][j] /= pivot

        # Update the list of elementary matrices
        elementary_matrix = [[1 if i == j else 0 for j in range(n)] for i in range(n)]
        for j in range(n):
            elementary_matrix[i][j] /= pivot
        elementary_matrices.append(elementary_matrix)

        # Perform row operations to eliminate the non-pivot elements
        for j in range(i + 1, n):
            factor = matrix_copy[j][i] / pivot
            for k in range(n):
                matrix_copy[j][k] -= factor * matrix_copy[i][k]
                inverse[j][k] -= factor * inverse[i][k]

    # Return the result, ensuring the original input matrix is preserved
    return {
        "input_matrix": matrix,
        "inverse_matrix": inverse,
        "elementary_matrices": elementary_matrices
    }


In [ ]:
matrix = [[2, 1, 1], [1, 3, 2], [1, 2, 4]]
result = find_inverse_with_elementary_matrices(matrix)

print(np.matmul(result.get('input_matrix'), result.get('inverse_matrix')))

[[0.82692308 0.30769231 0.38461538]
 [0.05384615 1.01538462 0.76923077]
 [0.00769231 0.43076923 1.53846154]]
